# <center> Cleaning eBay car sale data </center>

In this post, I will use Python to clean a eBay car sale dataset. The aim of this cleaning process is to provide a ready-to-use dataset for further analyses.

The eBay car sale dataset could be found [here](https://github.com/mark-hiep-dang/ebay-car-sale/blob/master/autos.csv). This dataset was sampled and dirtied by [*DataQuest*](http://dataquest.io) for the purpose of practicing.

>The cleaning tasks that were done in this project:
* Changing columns's names from camelcase to snakecase format; shorten some columns' names
* Convert some columns from string to numeric type (for example: price, odometer) 
* Working with date values.
* Mapping the categorical name from German to English.
* Spot out abnormal data

> Summary of what have been achieved with this project:
* Renamed the columns
* Converted the price and odometer columns from string to numeric
* Created three new integer columns representing dates: crawled_date, last_seen_date, ad_created_date.
* Deleted these columns because they only contain one value: seller, nr_of_pictures
* Translated categories name from German to English for these columns: abtest, vehicle_type, gearbox, fuel_type, unrepaired_damage
* Mapped nan value to others category for these columns: vehicle_type, fuel_type
* Spotted abnormal values in these columns (that should be paid attention to when performing further analy:
 * ***price***: range from 0 to 100 million dollar
 * ***registration_year***: range from 1000 to 9999. Set the registration of the records that have registration_year outside of the range 1886 to 2016 to nan.
 * ***registration_month***: contains month 0. Change this value to nan
 * ***power_ps***: range from 0 to 17,700ps (Result from Google show that the strongest cars these days have power of around 1,500ps


### Import and preview the dataset

In [89]:
import pandas as pd
import numpy as np
import re

In [90]:
autos = pd.read_csv("autos.csv", encoding = "Latin-1")

In [91]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [92]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

> There are five columns in the autos df with null values: vehicleType, gearbox, model, fuelType, and notRepairedDamage. The column with the largest missing rate is notRepariedDamange with ~20% missing data.

### Working with columns' names

> Transform the columns' names from camelcase to snakecase for convenience.

In [93]:
def camelcase_to_snakecase(s):
    s = re.sub(r"([a-z])([A-Z])", r"\1 \2", s).replace(" ", "_").lower()
    return s

In [94]:
new_columns = []
for c in autos.columns:
    new_columns.append(camelcase_to_snakecase(c))

In [95]:
autos.columns = new_columns

In [96]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'year_of_registration', 'gearbox', 'power_ps', 'model',
       'odometer', 'month_of_registration', 'fuel_type', 'brand',
       'not_repaired_damage', 'date_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

> Changing names of some columns for shorter ones.

In [97]:
autos.rename(columns = {"year_of_registration":"registration_year",
                        "month_of_registration":"registration_month",
                        "not_repaired_damage":"unrepaired_damage",
                        "date_created":"ad_created"}, inplace = True)

In [98]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [99]:
autos.describe()

,registration_year,power_ps,registration_month,nr_of_pictures,postal_code
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


>**nr_of_pictures** only has one value 0.0 => This should be dropped from the data frame.  
>**registration_year** contains value range from 1000 to 9999 that does not make sense => should investigate this column.  
>**registration_month** contains min value of 0 => should investigate these cases.  
>**power_pas contains** min value of 0. It is not reasonable to have a car with zero power => 
should investigate these cases.

In [100]:
autos.describe(include=np.object)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,gearbox,model,odometer,fuel_type,brand,unrepaired_damage,ad_created,last_seen
count,50000,50000,50000,50000,50000,50000,44905,47320,47242,50000,45518,50000,40171,50000,50000
unique,48213,38754,2,2,2357,2,8,2,245,13,7,40,2,76,39481
top,2016-03-30 17:37:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,manuell,golf,"150,000km",benzin,volkswagen,nein,2016-04-03 00:00:00,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,36993,4024,32424,30107,10687,35232,1946,8


> Should investigate these columns:
* **seller**: only 2 unique values and only 1/50000 cases is not *privat*
* **price** contains 1421 cases with zero value and should be converted to numbers.
* **abtest** only has 2 unique values. It could be binary yes/no value
* **odometer** should be converted to numbers

### Convert the price and odometer columns from string to numeric

In [101]:
autos["price"].unique()

array(['$5,000', '$8,500', '$8,990', ..., '$385', '$22,200', '$16,995'],
      dtype=object)

In [102]:
# Remove "$"" and base ten comma "," from the number and cast price to int
autos["price"] = autos["price"].str.replace("$","").str.replace(",", "").astype(int)

In [103]:
autos["odometer"].unique()

array(['150,000km', '70,000km', '50,000km', '80,000km', '10,000km',
       '30,000km', '125,000km', '90,000km', '20,000km', '60,000km',
       '5,000km', '100,000km', '40,000km'], dtype=object)

In [104]:
# Remove "km" and base ten comma "," from odometer and cast it to int
autos["odometer"] = autos["odometer"].str.replace("km", "").str.replace(",", "").astype(int)

In [105]:
# Rename odometer to odometer_km
autos.rename(columns = {"odometer":"odometer_km"}, inplace = True)

### Working with date columns

> Below are 5 columns that should represend date values:

In [106]:
autos[["date_crawled", "last_seen", "ad_created", "registration_month", "registration_year"]].head()

,date_crawled,last_seen,ad_created,registration_month,registration_year
0,2016-03-26 17:47:46,2016-04-06 06:45:54,2016-03-26 00:00:00,3,2004
1,2016-04-04 13:38:56,2016-04-06 14:45:08,2016-04-04 00:00:00,6,1997
2,2016-03-26 18:57:24,2016-04-06 20:15:37,2016-03-26 00:00:00,7,2009
3,2016-03-12 16:58:10,2016-03-15 03:16:28,2016-03-12 00:00:00,6,2007
4,2016-04-01 14:38:50,2016-04-01 14:38:50,2016-04-01 00:00:00,7,2003


> **date_crawled**, **last_seen**, **ad_created** have same format for date: ***yyyy-mm-dd hh:mm:ss***

> In order to quatitatively assess these date, it is better to convert them to numeric date. For example "2016-04-06" becomes 20169406

In [107]:
crawled_date = autos["date_crawled"].str[:10].str.replace("-", "").astype(int)

In [108]:
autos["crawled_date"] = crawled_date

In [109]:
last_seen_date = autos["last_seen"].str[:10].str.replace("-", "").astype(int)

In [110]:
autos["last_seen_date"] = last_seen_date

In [111]:
ad_created_date = autos["ad_created"].str[:10].str.replace("-", "").astype(int)

In [112]:
autos["ad_created_date"] = ad_created_date

In [113]:
autos[["crawled_date", "last_seen_date", "ad_created_date"]].head()

,crawled_date,last_seen_date,ad_created_date
0,20160326,20160406,20160326
1,20160404,20160406,20160404
2,20160326,20160406,20160326
3,20160312,20160315,20160312
4,20160401,20160401,20160401


### Categorical Columns

In [114]:
# Make a copy of the dataset before moving on
autos_copy = autos.copy()

> This is the list of categorical columns that should be examined: **seller**, **abtest**, **vehicle_type**, **gearbox**, **model**, **fuel_type**, **unrepaired_damage**

#### seller

In [115]:
autos["seller"].value_counts()

privat        49999
gewerblich        1
Name: seller, dtype: int64

> ***privat*** is ***private***; ***geweblich*** is ***commercial***

In [116]:
# Checking the only one record that seller is commercial
autos.loc[autos["seller"]=="gewerblich"]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,...,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen,crawled_date,last_seen_date,ad_created_date
7738,2016-03-15 18:06:22,Verkaufe_mehrere_Fahrzeuge_zum_Verschrotten,gewerblich,Angebot,100,control,kombi,2000,manuell,0,...,benzin,renault,NaN,2016-03-15 00:00:00,0,65232,2016-04-06 17:15:37,20160315,20160406,20160315


> There is only 1 over 50,000 record with a commercial seller => better to remove this column from the dataset

In [117]:
autos = autos.drop(labels = "seller", axis = 1)

#### abtest

In [118]:
autos["abtest"].value_counts(dropna = False)

test       25756
control    24244
Name: abtest, dtype: int64

> There are only two values for the **abtest** column: test and control. We can leave it there or convert to binary columns if needed later.

#### vehicle_type

In [119]:
autos["vehicle_type"].value_counts(dropna = False)

limousine     12859
kleinwagen    10822
kombi          9127
NaN            5095
bus            4093
cabrio         3061
coupe          2537
suv            1986
andere          420
Name: vehicle_type, dtype: int64

> There are about 10% of NaN data in this column. These should be replace by "others"

In [120]:
# Translate the categories in vehicle_type column to English
mapping_dict = {
    "limousine": "limousine",
    "kleinwage": "small car",
    "kombi": "station wagon",
    "bus": "bus",
    "cabrio": "convertible",
    "coupe": "coupe",
    "suv": "suv",
    "andere": "others",
    np.nan: "others"
}

autos["vehicle_type"] = autos["vehicle_type"].map(mapping_dict, na_action = "ignore")

#### gearbox

In [121]:
autos["gearbox"].value_counts(dropna = False)

manuell      36993
automatik    10327
NaN           2680
Name: gearbox, dtype: int64

> There are 2680 NaN data rows within "gearbox" column

In [122]:
# Translate the categories in "gearbox"
mapping_dict = {
    "manuell": "manual",
    "automatik": "automatic"
}

In [123]:
autos["gearbox"] = autos["gearbox"].map(mapping_dict, na_action="ignore")

#### model

In [124]:
autos["model"].value_counts(dropna = False).shape[0]

246

> There are 246 categories in the ***model*** column. This is because model was named differently by each brand. There is no need to transform this column.

#### fuel_type

In [125]:
autos["fuel_type"].value_counts(dropna = False)

benzin     30107
diesel     14567
NaN         4482
lpg          691
cng           75
hybrid        37
andere        22
elektro       19
Name: fuel_type, dtype: int64

> The NaN value should be replaced by "others" type

In [126]:
# Translate the fuel_type column
mapping_type = {
    "benzin": "petrol",
    "diesel": "diesel",
    np.nan: "others",
    "lpg": "lpg",
    "cng": "cng",
    "hybrid": "hybrid",
    "andere": "others",
    "elektro": "electro"
}

In [127]:
autos["fuel_type"] = autos["fuel_type"].map(mapping_type)

#### unrepaired_damage

In [129]:
autos["unrepaired_damage"].value_counts(dropna = False)

nein    35232
NaN      9829
ja       4939
Name: unrepaired_damage, dtype: int64

In [131]:
mapping_dict = {
    "nein": "no",
    "ja": "yes"
}

In [132]:
autos["unrepaired_damage"] = autos["unrepaired_damage"].map(mapping_dict, na_action="ignore")

### Explore some suspected columns

#### Price

In [133]:
# The number of unique value
autos["price"].unique().shape[0]

2357

In [134]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [146]:
# The number of cars that cheaper than 50 dollar
autos.loc[autos["price"] < 50].shape[0]

1639

In [140]:
# The number of cars that cost more than 50,000 dollar
autos.loc[autos["price"] > 50000].shape[0]

200

> The price range is from 0 to 100 million dollar.  
It is suspected that some car are dirt cheap and some are very expensive.  
We do not have any clue yet about if it is correct to have those price listed. So, let's keep it for now.

#### Odometer

In [23]:
autos["odometer_km"].unique().shape[0]

13

In [24]:
autos["odometer_km"].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [148]:
autos["odometer_km"].value_counts(dropna = False).sort_index()

5000        967
10000       264
20000       784
30000       789
40000       819
50000      1027
60000      1164
70000      1230
80000      1436
90000      1757
100000     2169
125000     5170
150000    32424
Name: odometer_km, dtype: int64

> Odometer seems not a correct number. This index could be classified in ranges.  
Besides, there is no abnormal data in this column

#### registration_year

In [29]:
autos["registration_year"].describe()

count    50000.000000
mean      2005.073280
std        105.712813
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

> The minimum value is 1000 that is even before cars were invented. The maximum value is 9999 that is too far in the future.  

> A car can not be registed after the listing was seen, any vehicle with a registration year above 2016 is inaccurate.  
The first modern car was invented in 1886 -> let's set the under bound of registration year to be 1886

In [150]:
reg_year = autos["registration_year"].between(1886, 2016)
reg_year = autos.loc[reg_year, "registration_year"]

In [151]:
reg_year.describe()

count    48028.00000
mean      2002.80351
std          7.31085
min       1910.00000
25%       1999.00000
50%       2003.00000
75%       2008.00000
max       2016.00000
Name: registration_year, dtype: float64

> There is more than 96% of data that has registration year between 1 and 2016. I will mark the other data as na

In [153]:
autos.loc[autos["registration_year"] < 1886, "registration_year"] = np.nan

In [154]:
autos.loc[autos["registration_year"] > 2016, "registration_year"] = np.nan

In [161]:
autos["registration_year"].isna().sum()

1972

#### nr_of_pictures

In [162]:
autos["nr_of_pictures"].value_counts(dropna = False)

0    50000
Name: nr_of_pictures, dtype: int64

> This column only contain 0 value -> This should be removed from the dataset

In [163]:
autos = autos.drop("nr_of_pictures", axis = 1)

#### registration_month

In [165]:
autos["registration_month"].value_counts(dropna = False).sort_index()

0     5075
1     3282
2     3008
3     5071
4     4102
5     4107
6     4368
7     3949
8     3191
9     3389
10    3651
11    3360
12    3447
Name: registration_month, dtype: int64

> There are 5075 records with registration_month 0. These should be replaced by NAN values.

In [166]:
autos.loc[autos["registration_month"] == 0, "registration_month"] = np.nan

#### power_ps

In [171]:
autos["power_ps"].describe()

count    50000.000000
mean       116.355920
std        209.216627
min          0.000000
25%         70.000000
50%        105.000000
75%        150.000000
max      17700.000000
Name: power_ps, dtype: float64

> ps stands for Pferdestärke in German that means hoursepower.  
In this dataset, car power range from 0 to 17,700 ps.  
A quick google shows that the strongest cars these days have power of around 1,500 ps and a regular car has more than 100ps power engine.

In [176]:
autos.loc[(autos["power_ps"] >= 100) & (autos["power_ps"] <= 1500),"power_ps"].describe()

count    29163.000000
mean       156.028529
std         65.527747
min        100.000000
25%        116.000000
50%        140.000000
75%        174.000000
max       1405.000000
Name: power_ps, dtype: float64

> We do not have any clue yet about if it is correct to limit that range of power. Let keep it for now

> The autos dataframe is now ready to save to further analysis